# Manipulating the Pandas DataFrame

The Jupyter notebook for this demo can be found in:
   - docs/quick_start/demo/op2_pandas_unstack.ipynb
   - https://github.com/SteveDoyle2/pyNastran/tree/master/docs/quick_start/demo/op2_pandas_unstack.ipynb
   
### This example will use pandas unstack
The unstack method on a DataFrame moves on index level from rows to columns.  First let's read in some data:

In [ ]:
import os
import pyNastran
pkg_path = pyNastran.__path__[0]
from pyNastran.op2.op2 import read_op2
import pandas as pd
pd.set_option('display.precision', 2)

op2_filename = os.path.join(pkg_path, '..', 'models', 'iSat', 'iSat_launch_100Hz.op2')
from pyNastran.op2.op2 import read_op2
isat = read_op2(op2_filename, build_dataframe=True, debug=False, skip_undefined_matrices=True)

In [ ]:
cbar = isat.op2_results.force.cbar_force[1].data_frame

In [ ]:
cbar.head()

First I'm going to pull out a small subset to work with

In [ ]:
csub = cbar.loc[3323:3324,1:2]
csub

I happen to like the way that's organized, but let's say that I want the have the item descriptions in columns and the mode ID's and element numbers in rows.  To do that, I'll first move the element ID's up to the columns using a .unstack(level=0) and the transpose the result:

In [ ]:
csub.unstack(level=0).T

unstack requires unique row indices so I can't work with CQUAD4 stresses as they're currently output, but I'll work with CHEXA stresses.  Let's pull out the first two elements and first two modes:

In [ ]:
chs = isat.op2_results.stress.chexa_stress[1].data_frame.loc[3684:3685,1:2]
chs

Now I want to put ElementID and the Node ID in the rows along with the Load ID, and have the items in the columns:

In [ ]:
cht = chs.unstack(level=[0,1]).T
cht

Maybe I'd like my rows organized with the modes on the inside.  I can do that by swapping levels:

We actually need to get rid of the extra rows using dropna():

In [ ]:
cht = cht.dropna()
cht

In [ ]:
# mode, eigr, freq, rad, eids, nids # initial
# nids, eids, eigr, freq, rad, mode # final

cht.swaplevel(0,4).swaplevel(1,5).swaplevel(2,5).swaplevel(4, 5)

Alternatively I can do that by first using reset_index to move all the index columns into data, and then using set_index to define the order of columns I want as my index:

In [ ]:
cht.reset_index().set_index(['ElementID','NodeID','Mode','Freq']).sort_index()